In [2]:
import os
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd

INPUT_DIR = '../data'
OUTPUT_DIR = 'outputs/'
SEED = 1230
# 初期設定
os.environ["PYTHONHASHSEED"] = str(SEED)
np.random.seed(SEED) # NOTE ソートのアルゴリズムがクイックソートのため、固定しないと順位が変わる可能性あり

In [3]:
# ログデータ
train_log = pd.read_csv(os.path.join(INPUT_DIR, 'train_log.csv'),)
train_label = pd.read_csv(os.path.join(INPUT_DIR, 'train_label.csv'),)
test_log = pd.read_csv(os.path.join(INPUT_DIR, 'test_log.csv'),)
# 宿データ
yado_df = pd.read_csv(os.path.join(INPUT_DIR, "yado.csv"))

In [4]:
def create_co_matrix(df: pd.DataFrame, df_: pd.DataFrame):
    df = df.merge(df_, how="inner", on="session_id")
    df = df.groupby(["yad_no_x", "yad_no_y"]).size().reset_index()
    df.columns = ["yad_no", "candidate_yad_no", "cnt"]
    return df

In [5]:
train_whole_df = pd.concat([train_log, train_label], axis=0, ignore_index=True,)
# 予約も含めた、閲覧共起行列の作成
visit_co_df_train = create_co_matrix(train_whole_df, train_whole_df)
visit_co_df_train.rename(columns={"cnt": "visit_cnt_train"}, inplace=True)

# テストの閲覧共起
visit_co_df_test = create_co_matrix(test_log, test_log)
visit_co_df_test.rename(columns={"cnt": "visit_cnt_test"}, inplace=True)

# trainの予約共起
reserve_co_df = create_co_matrix(train_log, train_label)
reserve_co_df.rename(columns={"cnt": "reserve_cnt"}, inplace=True)

In [7]:
RESERVE_STRENGTH = 10
TEST_CO_STRENGTH = 75 # NOTE 30以上であればほとんど変わらない？
# 学習データとテストデータの共起を外部結合
co_df = visit_co_df_train.merge(visit_co_df_test, how="outer",on=["yad_no", "candidate_yad_no"]).fillna(0)
# 予約結合
co_df = co_df.merge(reserve_co_df, how="left", on=["yad_no", "candidate_yad_no"]).fillna(0)
# 重みづけ
co_df["score"] = co_df["visit_cnt_train"] + co_df["reserve_cnt"] * RESERVE_STRENGTH # TODO 分布の値を用いて変動的にしたい
co_df["score"] += co_df["visit_cnt_test"] * TEST_CO_STRENGTH # TODO 分布の値を用いて変動的にしたい
# SELECT
co_df = co_df[["yad_no", "candidate_yad_no", "score"]]

In [19]:
COL = "sml_cd" # TODO ここ雑
additional_co_df = co_df.query("yad_no == candidate_yad_no").drop(columns=["candidate_yad_no"])
# 地域情報紐付け
additional_co_df = additional_co_df.merge(yado_df[["yad_no", COL]])
# 地域ごとのスコアでランクづけ
additional_co_df[f"{COL}_rank"] = additional_co_df.groupby(COL).score.rank(method="first", ascending=False).astype(int)
additional_co_df = additional_co_df.query(f"{COL}_rank < 10")
additional_co_df = yado_df[["yad_no", COL]].merge(additional_co_df, how="left", on=COL)
additional_co_df.rename(columns={"yad_no_x": "yad_no", "yad_no_y": "candidate_yad_no"}, inplace=True)
additional_co_df["score"] = 10 - additional_co_df[f"{COL}_rank"]
# SELECT
additional_co_df = additional_co_df[["yad_no", "candidate_yad_no", "score"]]

In [21]:
co_df

,yad_no,candidate_yad_no,score
0,1,1,2251.0
1,1,6431,1.0
2,2,2,461.0
3,2,36,12.0
4,2,217,11.0
...,...,...,...
266807,13798,13438,150.0
266808,13804,2983,75.0
266809,13804,4226,225.0
266810,13804,12835,75.0


In [22]:
# 最終的に使用するスコア行列
final_co_df = pd.concat([co_df, additional_co_df], axis=0).groupby(["yad_no", "candidate_yad_no"]).score.max().reset_index()

In [24]:
# セッション内閲覧は候補より上位にするため
SEQ_NO_STRENGTH = final_co_df.score.max()

def predict(df: pd.DataFrame) -> pd.DataFrame:
    # 後で最後の閲覧を消すため
    last_views = df.groupby("session_id").yad_no.last().rename("last_view_yad_no")
    # 後でセッション内の閲覧を上位にするため
    unique_views = df.groupby(["session_id", "yad_no"]).seq_no.min().reset_index()
    # スコア行列マージ
    df_ = df.merge(final_co_df, how="left", on="yad_no")
    # ランキング
    df_ = df_.groupby(["session_id", "candidate_yad_no"]).score.sum().reset_index()
    df_["score_rank"] = df_.groupby("session_id").score.rank(ascending=False, method="first")
    # スコアTOP15に絞る（計算量減らすため）
    df_ = df_.query("score_rank <= 15")
    # セッション内を上位に
    df_ = df_.merge(unique_views, how="left", left_on=["session_id", "candidate_yad_no"], right_on=["session_id", "yad_no"])
    df_.loc[df_.seq_no.notnull(), "score"] += SEQ_NO_STRENGTH 
    # 最後に閲覧の宿を消す
    df_ = df_.merge(last_views, how="left", on="session_id")
    df_ = df_.query("candidate_yad_no != last_view_yad_no")
    # 最終ランキング
    df_["score_rank"] = df_.groupby("session_id").score.rank(ascending=False, method="first")
    # TOP10を提示
    df_ = df_.query("score_rank <= 10").groupby("session_id").head(10)
    # submit形式に修正
    return df_.pivot_table(index="session_id", columns="score_rank", values="candidate_yad_no").fillna(0).astype(int)